In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
import pandas as pd
from torch.optim.lr_scheduler import CosineAnnealingLR
import copy
import time 
import torch.nn.functional as F

# Set random seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set default tensor type to float32
torch.set_default_tensor_type(torch.FloatTensor)

# ----------------------------------------------------------------------
# Normalization
# ----------------------------------------------------------------------

class Normalizer:
    def __init__(self, x_min, x_max):
        self.x_min = x_min
        self.x_max = x_max
        
    def normalize(self, x):
        return 2.0 * (x - self.x_min) / (self.x_max - self.x_min) - 1.0
        
    def denormalize(self, x_norm):
        return 0.5 * (x_norm + 1.0) * (self.x_max - self.x_min) + self.x_min

# ----------------------------------------------------------------------
# ANN
# ----------------------------------------------------------------------

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(2, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.hidden3 = nn.Linear(128, 128)
        self.hidden4 = nn.Linear(128, 128)  
        self.hidden5 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 2)
    def forward(self, x):
        x = torch.tanh(self.hidden1(x))
        x = torch.tanh(self.hidden2(x))
        x = torch.tanh(self.hidden3(x))
        x = torch.tanh(self.hidden4(x))
        x = torch.tanh(self.hidden5(x))
        return self.output(x)  # return [u, v]
    # Define the problem domain using the given vertices

# ----------------------------------------------------------------------
# Vertices
# ----------------------------------------------------------------------

vertices = np.array([
    [0, 0],
    [0, 4],
    [5, 4],
    [5, 0],
    [0, 0], # Closing the polygon
],dtype=np.float32)
path = Path(vertices)
def in_domain(x, y):
    points = np.column_stack((x.cpu().numpy(), y.cpu().numpy()))
    return torch.tensor(path.contains_points(points), dtype=torch.bool, device=device)

# ----------------------------------------------------------------------
# BC
# ----------------------------------------------------------------------

# Define boundary conditions with tolerance
def BC_bottom(x, y):
    tol = 1e-6
    return ((torch.abs(y - 0) < tol) & (x >= 0) & (x <= 5)).squeeze()

def BC_left(x, y):
    tol = 1e-6
    return ((torch.abs(x - 0) < tol) & (y >= 0) & (y <= 4)).squeeze()

def BC_top(x, y):
    tol = 1e-6
    return ((torch.abs(y - 4) < tol) & (x >= 0) & (x <= 5)).squeeze()

def BC_right(x, y):
    tol = 1e-6
    return ((torch.abs(x - 5) < tol) & (y >= 0) & (y <= 4)).squeeze()

# ----------------------------------------------------------------------
# BC Loss (Modified for Inverse Problem)
# ----------------------------------------------------------------------

def BC(xy, net, E, nu): # E and nu are now passed as arguments
    x, y = xy[:, 0].unsqueeze(1), xy[:, 1].unsqueeze(1)
    
    output = net(xy)
    u = output[:, 0:1]  # Split output channel (u)
    v = output[:, 1:2]  # Split output channel (v)
    # Get boundary conditions
    
    bc_b = BC_bottom(x, y)
    bc_l = BC_left(x, y)
    bc_t = BC_top(x, y)
    bc_r = BC_right(x, y)
    
    
    loss = F.mse_loss(u[bc_b], torch.zeros_like(u[bc_b])) + F.mse_loss(v[bc_b], torch.zeros_like(v[bc_b]))  # ux = 0, uy = 0 at bottom
    loss += F.mse_loss(u[bc_l], torch.zeros_like(u[bc_l]))  # ux = 0 at left
    loss += F.mse_loss(u[bc_r], torch.zeros_like(u[bc_r]))  # ux = 0 at right

    
    # Process top boundary
    xy_top = xy[bc_t].requires_grad_(True)
    x_top = xy_top[:, 0:1]  # x-coordinates of points on the top boundary
    
    output_top = net(xy_top)
    u_top = output_top[:, 0:1]
    v_top = output_top[:, 1:2]
    
    u_x_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 0]
    u_y_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 1]
    v_x_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 0]
    v_y_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 1]
    
    
    # Calculate stresses (Plane Strain formulation)
    sigma_xx_top = (E / ((1 + nu) * (1 - 2 * nu))) * ((1 - nu) * u_x_top + nu * v_y_top)
    sigma_yy_top = (E / ((1 + nu) * (1 - 2 * nu))) * (nu * u_x_top + (1 - nu) * v_y_top)
    sigma_xy_top = E / (2 * (1 + nu)) * (u_y_top + v_x_top)
    
    # Create mask for points with distributed load (x from 2 to 3)
    load_mask = (x_top >= 2) & (x_top <= 3)
    load_mask = load_mask.squeeze()  # Remove extra dimension
    
    # All points on top have sigma_xy = 0
    loss += F.mse_loss(sigma_xy_top, torch.zeros_like(sigma_xy_top))

    # Add condition for the area without load on the top edge
    no_load_mask = ~load_mask
    if torch.any(no_load_mask):
 
        loss += F.mse_loss(sigma_yy_top[no_load_mask], torch.zeros_like(sigma_yy_top[no_load_mask]))
    
    # Points with load have sigma_yy = -0.5 (negative for compression)
    if torch.any(load_mask):
        loss += F.mse_loss(sigma_yy_top[load_mask], -0.5 * torch.ones_like(sigma_yy_top[load_mask]))
    
    # Process right boundary
    xy_right = xy[bc_r].requires_grad_(True)
    output_right = net(xy_right)
    u_right = output_right[:, 0:1]
    v_right = output_right[:, 1:2]
    
    u_y_right = torch.autograd.grad(u_right.sum(), xy_right, create_graph=True)[0][:, 1]
    v_x_right = torch.autograd.grad(v_right.sum(), xy_right, create_graph=True)[0][:, 0]
    u_x_right = torch.autograd.grad(u_right.sum(), xy_right, create_graph=True)[0][:, 0]
    v_y_right = torch.autograd.grad(v_right.sum(), xy_right, create_graph=True)[0][:, 1]
    sigma_xy_right = E / (2 * (1 + nu)) * (u_y_right + v_x_right)
    loss += F.mse_loss(sigma_xy_right, torch.zeros_like(sigma_xy_right)) # Sigma_xy = 0 at right
    # Process left boundary
    xy_left = xy[bc_l].requires_grad_(True)
    output_left = net(xy_left)
    u_left = output_left[:, 0:1]
    v_left = output_left[:, 1:2]
    
    u_y_left = torch.autograd.grad(u_left.sum(), xy_left, create_graph=True)[0][:, 1]
    v_x_left = torch.autograd.grad(v_left.sum(), xy_left, create_graph=True)[0][:, 0]
    
    sigma_xy_left = E / (2 * (1 + nu)) * (u_y_left + v_x_left)
    loss += F.mse_loss(sigma_xy_left, torch.zeros_like(sigma_xy_left)) # Sigma_xy = 0 at left

    return loss

# ----------------------------------------------------------------------
# Datapoint sampling (for PDE collocation points)
# ----------------------------------------------------------------------

def generate_point_distribution(nx=100, ny=100, load_density=10, load_influence=70):
    # Defined boundaries of the domain
    x_min, x_max = 0.0, 5.0
    y_min, y_max = 0.0, 4.0
    
    # Create random points in the domain
    # Add number of points to be 4 times the grid size
    num_random_points = nx * ny * 4
    x_random = torch.rand(num_random_points, device=device) * (x_max - x_min) + x_min
    y_random = torch.rand(num_random_points, device=device) * (y_max - y_min) + y_min
    
    # print(f"Total random points generated: {len(x_random)}")
    
    load_x_start, load_x_end = 2.0, 3.0
    load_y = 4.0
    
    distances = torch.zeros_like(x_random)
    
   
    in_x_range = (x_random >= load_x_start) & (x_random <= load_x_end)
    
    distances[in_x_range] = torch.abs(y_random[in_x_range] - load_y)
    
     
    left_of_range = x_random < load_x_start
    
    distances[left_of_range] = torch.sqrt(
        (x_random[left_of_range] - load_x_start)**2 + 
        (y_random[left_of_range] - load_y)**2
    )
    
    
    right_of_range = x_random > load_x_end
    
    distances[right_of_range] = torch.sqrt(
        (x_random[right_of_range] - load_x_end)**2 + 
        (y_random[right_of_range] - load_y)**2
    )
    
    base_prob = 0.2 
    additional_prob = load_influence / (1 + (distances * load_density)**2)
    
    
    keep_prob = torch.clamp(base_prob + additional_prob, 0, 1)
    
    
    keep_mask = torch.rand(num_random_points, device=device) < keep_prob
    
    
    x_final = x_random[keep_mask]
    y_final = y_random[keep_mask]
    
    # print(f"Final points after density-based filtering: {len(x_final)}")
    
    
    plt.figure(figsize=(10, 8))
    
    
    plt.scatter(x_final.cpu().numpy(), y_final.cpu().numpy(), s=1, color='blue', alpha=0.7)
    
    
    plt.plot([load_x_start, load_x_end], [load_y, load_y], 'r-', linewidth=2, label='Load line')
    
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Point Distribution with Enhanced Load Area Density - {len(x_final)} points')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.gca().set_aspect('equal')
    plt.legend()
    
    plt.savefig('collocation_points.png')
    print("Collocation point plot saved to collocation_points.png")
    plt.close()
    
    return x_final, y_final

# ----------------------------------------------------------------------
# PDE (Modified for Inverse Problem)
# ----------------------------------------------------------------------

def PDE(x, y, net, E, nu): # E and nu are now passed as arguments
    # Reshape 1D tensors into column vectors for concatenation
    x_reshaped = x.unsqueeze(1) if x.dim() == 1 else x
    y_reshaped = y.unsqueeze(1) if y.dim() == 1 else y
    
    xy = torch.cat([x_reshaped, y_reshaped], dim=1)
    xy.requires_grad = True
    
    output = net(xy)
    u = output[:, 0:1]
    v = output[:, 1:2]
    
    u_x = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    u_y = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    v_x = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    v_y = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    
    gamma = 0 # Distributed load (Mn/m^2)
    
    
    C1 = E / ((1 + nu) * (1 - 2 * nu))
    sigma_xx = C1 * ((1 - nu) * u_x + nu * v_y)
    sigma_yy = C1 * (nu * u_x + (1 - nu) * v_y)
    sigma_xy = E / (2 * (1 + nu)) * (u_y + v_x)
    
    f_x = torch.zeros_like(x_reshaped)
    f_y = -gamma * torch.ones_like(y_reshaped)
    
    R_x = torch.autograd.grad(sigma_xx.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_x
    R_y = torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_yy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_y

    loss_x = F.mse_loss(R_x, torch.zeros_like(R_x))
    loss_y = F.mse_loss(R_y, torch.zeros_like(R_y))
    
    return loss_x, loss_y

# ----------------------------------------------------------------------
# Data Loading and Loss Functions
# ----------------------------------------------------------------------

def load_data(filename, device):
    """Loads data from FEM_PDE.csv, handling separate coordinates for displacement and stress."""
    try:
        data = pd.read_csv(filename)
    except FileNotFoundError:
        print(f"Error: Data file '{filename}' not found.")
        print("Please make sure 'FEM_PDE.csv' is in the same directory as the script.")
        return None

    # --- 1. Process Displacement Data (X, Y) ---
    disp_cols = ['X', 'Y', 'ux', 'uy']
    if not all(col in data.columns for col in disp_cols):
        print(f"Error: Missing displacement columns. Need: {', '.join(disp_cols)}")
        return None
    
    # Drop NaNs *only* from displacement data
    disp_data = data[disp_cols].dropna()
    if len(disp_data) == 0:
        print("Error: No valid displacement data found.")
        return None

    X_disp = torch.tensor(disp_data['X'].values, dtype=torch.float32).unsqueeze(1).to(device)
    Y_disp = torch.tensor(disp_data['Y'].values, dtype=torch.float32).unsqueeze(1).to(device)
    xy_disp_data = torch.cat([X_disp, Y_disp], dim=1)
    u_data = torch.tensor(disp_data['ux'].values, dtype=torch.float32).unsqueeze(1).to(device)
    v_data = torch.tensor(disp_data['uy'].values, dtype=torch.float32).unsqueeze(1).to(device)
    print(f"Loaded {len(xy_disp_data)} displacement data points.")

    # --- 2. Process Stress Data (X1, Y1) ---
    stress_cols = ['X1', 'Y1', 'sigma_xx', 'sigma_yy', 'sigma_xy']
    if not all(col in data.columns for col in stress_cols):
        print(f"Error: Missing stress columns. Need: {', '.join(stress_cols)}")
        return None
        
    # Drop NaNs *only* from stress data
    stress_data = data[stress_cols].dropna()
    if len(stress_data) == 0:
        print("Error: No valid stress data found.")
        return None

    X_stress = torch.tensor(stress_data['X1'].values, dtype=torch.float32).unsqueeze(1).to(device)
    Y_stress = torch.tensor(stress_data['Y1'].values, dtype=torch.float32).unsqueeze(1).to(device)
    xy_stress_data = torch.cat([X_stress, Y_stress], dim=1)
    
    # Convert stresses from kPa (data) to MPa (model unit)
    s_xx_data = torch.tensor(stress_data['sigma_xx'].values / 1000.0, dtype=torch.float32).unsqueeze(1).to(device)
    s_yy_data = torch.tensor(stress_data['sigma_yy'].values / 1000.0, dtype=torch.float32).unsqueeze(1).to(device)
    s_xy_data = torch.tensor(stress_data['sigma_xy'].values / 1000.0, dtype=torch.float32).unsqueeze(1).to(device)
    print(f"Loaded {len(xy_stress_data)} stress data points.")

    # --- 3. Return all 7 tensors ---
    return xy_disp_data, u_data, v_data, xy_stress_data, s_xx_data, s_yy_data, s_xy_data


def data_loss(net, xy_disp_data, u_data, v_data, xy_stress_data, s_xx_data, s_yy_data, s_xy_data, E, nu):
    """Calculates the MSE loss from separate displacement and stress datasets."""
    
    # --- 1. Displacement Loss (at X, Y coordinates) ---
    output_disp = net(xy_disp_data)
    u_pred = output_disp[:, 0:1]
    v_pred = output_disp[:, 1:2]
    
    loss_u = F.mse_loss(u_pred, u_data)
    loss_v = F.mse_loss(v_pred, v_data)

    # --- 2. Stress Loss (at X1, Y1 coordinates) ---
    # We must enable gradients for this pass
    xy_stress_data.requires_grad_(True)
    output_stress = net(xy_stress_data)
    u_stress = output_stress[:, 0:1]
    v_stress = output_stress[:, 1:2]

    # Calculate predicted stresses using autodiff
    u_x = torch.autograd.grad(u_stress.sum(), xy_stress_data, create_graph=True)[0][:, 0].unsqueeze(1)
    u_y = torch.autograd.grad(u_stress.sum(), xy_stress_data, create_graph=True)[0][:, 1].unsqueeze(1)
    v_x = torch.autograd.grad(v_stress.sum(), xy_stress_data, create_graph=True)[0][:, 0].unsqueeze(1)
    v_y = torch.autograd.grad(v_stress.sum(), xy_stress_data, create_graph=True)[0][:, 1].unsqueeze(1)

    # Use the consistent plane strain formula
    C1 = E / ((1 + nu) * (1 - 2 * nu))
    s_xx_pred = C1 * ((1 - nu) * u_x + nu * v_y)
    s_yy_pred = C1 * (nu * u_x + (1 - nu) * v_y)
    s_xy_pred = E / (2 * (1 + nu)) * (u_y + v_x)

    # Calculate MSE loss for all stress components
    loss_s_xx = F.mse_loss(s_xx_pred, s_xx_data)
    loss_s_yy = F.mse_loss(s_yy_pred, s_yy_data)
    loss_s_xy = F.mse_loss(s_xy_pred, s_xy_data)
    
    # --- 3. Total Data Loss ---
    total_data_loss = loss_u + loss_v + loss_s_xx + loss_s_yy + loss_s_xy
    
    return total_data_loss

# ----------------------------------------------------------------------
# Training (Modified for Inverse Problem)
# ----------------------------------------------------------------------

def train(net, optimizer_net, optimizer_material, E_param, nu_param, data_tensors, n_epochs, data_loss_weight=1.0, nx=100, ny=100):
    scheduler_net = CosineAnnealingLR(optimizer_net, T_max=n_epochs, eta_min=1e-4)
    scheduler_material = CosineAnnealingLR(optimizer_material, T_max=n_epochs, eta_min=1e-2)
    
    best_loss = float('inf')
    patience = 0
    max_patience = 3000
    
    prev_net = None
    
    # Add lists for loss history
    loss_history = []
    pde_x_history = []
    pde_y_history = []
    bc_history = []
    data_history = []
    epoch_history = []
    E_history = []  # NEW: Track E values
    nu_history = []  # NEW: Track nu values
    
    # Create training points (collocation points) only once
    print("Generating PDE collocation points...")
    x_inner, y_inner = generate_point_distribution(nx=nx, ny=ny)
    print(f"Generated {len(x_inner)} interior points")
    
    # Create boundary points only once
    print("Generating boundary points...")
    num_boundary_points = 1000
    
    # Create points on the bottom edge (y=0)
    x_bottom = torch.linspace(0, 5, num_boundary_points, device=device)
    y_bottom = torch.zeros(num_boundary_points, device=device)
    
    # Create points on the left edge (x=0)
    x_left = torch.zeros(num_boundary_points, device=device)
    y_left = torch.linspace(0, 4, num_boundary_points, device=device)
    
    # Create points on the top edge (y=4)
    x_top = torch.linspace(0, 5, num_boundary_points, device=device)
    y_top = torch.ones(num_boundary_points, device=device) * 4
    
    # Create points on the right edge (x=5)
    x_right = torch.ones(num_boundary_points, device=device) * 5
    y_right = torch.linspace(0, 4, num_boundary_points, device=device)
    
    # Total boundary points
    x_b = torch.cat([x_bottom, x_left, x_top, x_right])
    y_b = torch.cat([y_bottom, y_left, y_top, y_right])
    
    # Create tensor for BC function
    xy_b = torch.stack([x_b, y_b], dim=1)
    print(f"Generated {len(xy_b)} boundary points")
    
    # Unpack data tensors
    xy_disp_data, u_data, v_data, xy_stress_data, s_xx_data, s_yy_data, s_xy_data = data_tensors
    print(f"Using {len(xy_disp_data)} displacement points and {len(xy_stress_data)} stress points.")
    
    print(f"Starting training for {n_epochs} epochs...")
    start_time = time.time()
    
    for epoch in range(n_epochs):
        optimizer_net.zero_grad()
        optimizer_material.zero_grad()
        
        # Constrain E and nu to physical values
        E = F.softplus(E_param) + 1.0
        nu = 0.01 + 0.48 * torch.sigmoid(nu_param)
        
        # 1. PDE Loss (Physics)
        loss_pde_x, loss_pde_y = PDE(x_inner, y_inner, net, E, nu)
        loss_pde = loss_pde_x + loss_pde_y
        
        # 2. BC Loss (Boundaries)
        loss_bc = BC(xy_b, net, E, nu)
        
        # 3. Data Loss (Observations)
        loss_data = data_loss(net, 
                              xy_disp_data, u_data, v_data, 
                              xy_stress_data, s_xx_data, s_yy_data, s_xy_data, 
                              E, nu)
        
        # Calculate total loss
        loss = loss_pde + loss_bc + data_loss_weight * loss_data
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.5)
        torch.nn.utils.clip_grad_norm_([E_param, nu_param], max_norm=0.1)
    
        optimizer_net.step()
        optimizer_material.step()
        scheduler_net.step()
        scheduler_material.step()
        
        # Save history (MODIFIED: Now includes E and nu)
        loss_history.append(loss.item())
        pde_x_history.append(loss_pde_x.item())
        pde_y_history.append(loss_pde_y.item())
        bc_history.append(loss_bc.item())
        data_history.append(loss_data.item())
        epoch_history.append(epoch + 1)  # Use epoch+1 for 1-based indexing
        E_history.append(E.item())  # NEW
        nu_history.append(nu.item())  # NEW
        
        if loss < best_loss:
            best_loss = loss
            patience = 0
            
            prev_net = copy.deepcopy(net)
            
            torch.save({
            'net_state_dict': net.state_dict(),
            'optimizer_net_state_dict': optimizer_net.state_dict(),
            'optimizer_material_state_dict': optimizer_material.state_dict(),
            'scheduler_net_state_dict': scheduler_net.state_dict(),
            'scheduler_material_state_dict': scheduler_material.state_dict(),
            'E_param': E_param.item(),
            'nu_param': nu_param.item(),
            'E_final': E.item(),
            'nu_final': nu.item(),
            'loss': loss,
            'epoch': epoch,
        }, 'best_model_inverse.pth')
                
        if (epoch + 1) % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.8f}, '
                  f'PDE: {loss_pde.item():.8f}, BC: {loss_bc.item():.8f}, Data: {loss_data.item():.8f}, '
                  f'LR_material: {scheduler_material.get_last_lr()[0]:.8f}, Time: {elapsed_time:.2f}s')
            print(f'    Estimated E: {E.item():.4f}, nu: {nu.item():.4f}')
            start_time = time.time()
    
    # NEW: Save training history to CSV
    training_df = pd.DataFrame({
        'Epoch': epoch_history,
        'Total_Loss': loss_history,
        'PDE_x_Loss': pde_x_history,
        'PDE_y_Loss': pde_y_history,
        'BC_Loss': bc_history,
        'Data_Loss': data_history,
        'E_MPa': E_history,
        'nu': nu_history,
        'Learning_Rate_Net': [scheduler_net.get_last_lr()[0]] * len(epoch_history),
        'Learning_Rate_Material': [scheduler_material.get_last_lr()[0]] * len(epoch_history) 
    })
    training_df.to_csv('training_history.csv', index=False)
    print("Training history saved to training_history.csv")
    
    # Create graph Loss history
    plt.figure(figsize=(12, 8))
    
    # Plot total loss
    plt.subplot(2, 1, 1)
    plt.plot(epoch_history, loss_history, 'b-', label='Total Loss')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (log scale)')
    plt.title('Training Loss History')
    plt.legend()
    plt.grid(True)
    
    # Plot component losses
    plt.subplot(2, 1, 2)
    plt.plot(epoch_history, pde_x_history, 'r-', label='PDE_x Loss')
    plt.plot(epoch_history, pde_y_history, 'g-', label='PDE_y Loss')
    plt.plot(epoch_history, bc_history, 'b-', label='BC Loss')
    plt.plot(epoch_history, data_history, 'm-', label=f'Data Loss (x{data_loss_weight})')
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Component Losses (log scale)')
    plt.title('Component Losses History')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('loss_history_inverse.png')
    plt.close()
    

    plt.figure(figsize=(12, 5))

# --- Plot E evolution ---
    plt.subplot(1, 2, 1)
    plt.plot(epoch_history, E_history, 'b-', linewidth=1.5, label='Predicted E')
    plt.axhline(y=15, color='k', linestyle='--', linewidth=1.2, label='True E = 15 MPa')  # เส้นประ
    plt.xlabel('Epoch')
    plt.ylabel("Young's Modulus E (MPa)")
    plt.title('Evolution of E during Training')
    plt.grid(False)
    plt.legend()

# --- Plot ν evolution ---
    plt.subplot(1, 2, 2)
    plt.plot(epoch_history, nu_history, 'r-', linewidth=1.5, label='Predicted ν')
    plt.axhline(y=0.3, color='k', linestyle='--', linewidth=1.2, label='True ν = 0.3')  # เส้นประ
    plt.xlabel('Epoch')
    plt.ylabel("Poisson's Ratio ν")
    plt.title('Evolution of ν during Training')
    plt.grid(False)
    plt.legend()

    plt.tight_layout()
    plt.savefig('material_parameters_evolution.png')
    plt.close()
    print("Material parameters evolution plot saved to material_parameters_evolution.png")

    
    # Additional return value for metrics for tracking training progress
    metrics = {
        'loss_history': loss_history,
        'pde_x_history': pde_x_history,
        'pde_y_history': pde_y_history,
        'bc_history': bc_history,
        'data_history': data_history,
        'epoch_history': epoch_history,
        'E_history': E_history,  
        'nu_history': nu_history  
    }
    
    # Load the best model state
    print("Loading best model from 'best_model_inverse.pth'")
    checkpoint = torch.load('best_model_inverse.pth')
    net.load_state_dict(checkpoint['net_state_dict'])
    final_E = checkpoint['E_final']
    final_nu = checkpoint['nu_final']
    
    print("Training completed.")
    return net, metrics, final_E, final_nu

# ----------------------------------------------------------------------
# Plot result (Modified for Inverse Problem)
# ----------------------------------------------------------------------

def plot_results(net, E, nu, output_filename='Footing_data_inverse.csv'): # E, nu passed as args
    nx, ny = 100, 100
    X = torch.linspace(0, 5.0, nx, device=device)  
    Y = torch.linspace(0, 4.0, ny, device=device)  
    
    xx, yy = torch.meshgrid(X, Y, indexing='ij')
    
    X = xx.flatten()
    Y = yy.flatten()
    
   
    mask = in_domain(X, Y)
    mask_np = mask.cpu().numpy()  
    
    
    XY = torch.stack([X, Y], dim=1)
    XY.requires_grad_(True)
    
    with torch.enable_grad():
        output = net(XY)
        U = output[:, 0:1]  
        V = output[:, 1:2]  
        
        U_x = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 0]
        U_y = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 1]
        V_x = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 0]
        V_y = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 1]
        
        
        # *** CORRECTION: Using Plane Strain equations ***
        C1 = E / ((1 + nu) * (1 - 2 * nu))
        sigma_xx = C1 * ((1 - nu) * U_x + nu * V_y)
        sigma_yy = C1 * (nu * U_x + (1 - nu) * V_y)
        sigma_xy = E / (2 * (1 + nu)) * (U_y + V_x)
    
    
    X_np = X.detach().cpu().numpy()
    Y_np = Y.detach().cpu().numpy()
    U_full = U.detach().cpu().numpy().squeeze()
    V_full = V.detach().cpu().numpy().squeeze()
    sigma_xx_full = sigma_xx.detach().cpu().numpy().squeeze()*1000 # Convert MPa to kPa for plotting
    sigma_yy_full = sigma_yy.detach().cpu().numpy().squeeze()*1000 # Convert MPa to kPa for plotting
    sigma_xy_full = sigma_xy.detach().cpu().numpy().squeeze()*1000 # Convert MPa to kPa for plotting
    
    
    magnitude = np.sqrt(U_full**2 + V_full**2)
    
    
    df_out = pd.DataFrame({
        'X': X_np[mask_np],
        'Y': Y_np[mask_np],
        'ux': U_full[mask_np],
        'uy': V_full[mask_np],
        'sigma_xx_kPa': sigma_xx_full[mask_np],
        'sigma_yy_kPa': sigma_yy_full[mask_np],
        'sigma_xy_kPa': sigma_xy_full[mask_np],
        'magnitude': magnitude[mask_np]
    })
    
    
    df_out.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")
    
    
    mask_cpu = mask.cpu().numpy()
    U_masked = np.ma.masked_array(U_full, mask=~mask_cpu)
    V_masked = np.ma.masked_array(V_full, mask=~mask_cpu)
    sigma_xx_masked = np.ma.masked_array(sigma_xx_full, mask=~mask_cpu)
    sigma_yy_masked = np.ma.masked_array(sigma_yy_full, mask=~mask_cpu)
    sigma_xy_masked = np.ma.masked_array(sigma_xy_full, mask=~mask_cpu)
    magnitude_masked = np.ma.masked_array(magnitude, mask=~mask_cpu)
    
    
    plt.figure(figsize=(16, 12))
    fig_title = f"Inverse Problem Results (Estimated E={E:.2f} MPa, $\\nu$={nu:.3f})"
    plt.suptitle(fig_title, fontsize=16)
    
    # Displacement plots
    plt.subplot(2, 3, 1)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), U_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Displacement ux (m)')
    plt.title('Displacement in x-direction')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    
    plt.subplot(2, 3, 2)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), V_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Displacement uy (m)')
    plt.title('Displacement in y-direction')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    
    plt.subplot(2, 3, 3)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), magnitude_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Displacement magnitude (m)')
    plt.title('Displacement magnitude')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Stress plots
    plt.subplot(2, 3, 4)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), sigma_xx_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Stress $\\sigma_{xx}$ (kPa)')
    plt.title('Normal stress $\\sigma_{xx}$')
    plt.xlabel('X')
    plt.ylabel('Y')
    
   
    plt.subplot(2, 3, 5)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), sigma_yy_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Stress $\\sigma_{yy}$ (kPa)')
    plt.title('Normal stress $\\sigma_{yy}$')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    
    plt.subplot(2, 3, 6)
    plt.pcolormesh(X_np.reshape(nx, ny), Y_np.reshape(nx, ny), sigma_xy_masked.reshape(nx, ny), shading='auto', cmap='jet')
    plt.colorbar(label='Stress $\\sigma_{xy}$ (kPa)')
    plt.title('Shear stress $\\sigma_{xy}$')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust for suptitle
    plt.savefig('results_plots_inverse.png', dpi=300)
    plt.close()
    print("Result plots saved to results_plots_inverse.png")

# ----------------------------------------------------------------------
# Main execution (Modified for Inverse Problem)
# ----------------------------------------------------------------------

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # --- 1. Load Data ---
    data_filename = 'FEM_PDE.csv'
    data_tensors = load_data(data_filename, device)
    
    if data_tensors is not None:
        print(f"Successfully loaded {len(data_tensors[0])} data points from {data_filename}")
        
        # --- 2. Initialize network ---
        net = Net().to(device)
        
        # --- 3. Initialize Trainable Material Parameters ---
        # We initialize them as unconstrained parameters.
        # The 'train' function will map them to valid ranges.
        
        # Initial guess for E: 15.0. 
        # We need to find 'x' such that softplus(x) + 1.0 = 15.0
        # softplus(x) = 14.0 -> log(1 + exp(x)) = 14.0 -> x approx 14.0
        initial_E_param = torch.tensor([-10.0], device=device, requires_grad=True)
        
        # Initial guess for nu: 0.3. 
        # We need to find 'x' such that 0.01 + 0.48 * sigmoid(x) = 0.3
        # sigmoid(x) = (0.3 - 0.01) / 0.48 = 0.604
        # x = log(0.604 / (1 - 0.604)) approx 0.42
        initial_nu_param = torch.tensor([-1.466], device=device, requires_grad=True)

        print(f"Initial E guess: {(F.softplus(initial_E_param) + 1.0).item():.4f} MPa")
        print(f"Initial nu guess: {(0.01 + 0.48 * torch.sigmoid(initial_nu_param)).item():.4f}")

        # --- 4. Optimizer ---
        # Add E and nu parameters to the optimizer
        optimizer_net = optim.Adam(net.parameters(), lr=0.001)
        optimizer_material = optim.Adam([initial_E_param, initial_nu_param], lr=0.05)
        
        # --- 5. Train the network ---
        # Set a data loss weight. This is a critical hyperparameter to tune.
        # If data loss is too low, increase weight. If PDE/BC losses are ignored, decrease it.
        data_weight = 300.0 
        
        trained_net, metrics, final_E, final_nu = train(
            net=net, 
            optimizer_net=optimizer_net,
            optimizer_material=optimizer_material,
            E_param=initial_E_param, 
            nu_param=initial_nu_param,
            data_tensors=data_tensors,
            n_epochs=30000, # You may need more epochs (e.g., 20k-50k)
            data_loss_weight=data_weight,
            nx=100, # Collocation points grid density
            ny=100
        )
        
        print("\n--- Inverse Analysis Complete ---")
        print(f"Estimated Young's Modulus (E): {final_E:.4f} MPa")
        print(f"Estimated Poisson's Ratio (nu): {final_nu:.4f}")
        
        # --- 6. Plot results ---
        try:
            plot_results(trained_net, final_E, final_nu)
        except Exception as e:
            print(f"Error in plot_results: {str(e)}")
    else:
        print("Exiting due to data loading error.")

Using device: cuda
Using device: cuda
Loaded 24675 displacement data points.
Loaded 19740 stress data points.
Successfully loaded 24675 data points from FEM_PDE.csv
Initial E guess: 1.0000 MPa
Initial nu guess: 0.1000


/venv/main/lib/python3.12/site-packages/torch/__init__.py:1264: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


Generating PDE collocation points...
Collocation point plot saved to collocation_points.png
Generated 17740 interior points
Generating boundary points...
Generated 4000 boundary points
Using 24675 displacement points and 19740 stress points.
Starting training for 30000 epochs...
Epoch 100/30000, Loss: 5.02255344, PDE: 0.02565440, BC: 0.05320664, Data: 0.01647897, LR_material: 0.04999890, Time: 4.04s
    Estimated E: 1.0149, nu: 0.4291
Epoch 200/30000, Loss: 3.58782959, PDE: 0.02657792, BC: 0.03608976, Data: 0.01175054, LR_material: 0.04999561, Time: 3.11s
    Estimated E: 1.2088, nu: 0.4402
